In [2]:
#!wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz

In [3]:
#import os
#!gunzip -c download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz >/tmp/lines

In [4]:
##os.mkdir ('/tmp/lines')
#!split -a 3 -l 100000 download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz /tmp/lines-

In [5]:
##BUCKET="your-bucket"
##gsutil -m cp -r lines gs://${BUCKET?}/opensubtitles/raw/""

In [6]:
! wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz

--2020-06-24 20:39:24--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3663376519 (3.4G) [application/gzip]
Saving to: ‘en.txt.gz’

en.txt.gz           100%[===================>]   3.41G  29.4MB/s    in 62s     

2020-06-24 20:40:27 (56.5 MB/s) - ‘en.txt.gz’ saved [3663376519/3663376519]



In [7]:
! apt install gunzip
! gunzip en.txt.gz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package gunzip


In [8]:
#The tail command, as the name implies, print the last N number of data of the given input. 
#By default it prints the last 10 lines of the specified files. 
# can refer to:  geeksforgeeks.org/tail-command-linux-examples/

!tail en.txt

- No, no, no, no!
- You killed my friend, my best friend, my lifelong friend!
I'm gonna kill every one of you!
- Get out of here!
Watch out!
No!
Oh, I've never been so glad to see anybody in my life.
What happened to him?
- He bled to death.
Come on, let's get the hell out of here.


In [9]:
!tail -7 en.txt

- Get out of here!
Watch out!
No!
Oh, I've never been so glad to see anybody in my life.
What happened to him?
- He bled to death.
Come on, let's get the hell out of here.


In [10]:
! split -a 3 -l 100000 --verbose en.txt /tmp/lines-

creating file '/tmp/lines-aaa'
creating file '/tmp/lines-aab'
creating file '/tmp/lines-aac'
creating file '/tmp/lines-aad'
creating file '/tmp/lines-aae'
creating file '/tmp/lines-aaf'
creating file '/tmp/lines-aag'
creating file '/tmp/lines-aah'
creating file '/tmp/lines-aai'
creating file '/tmp/lines-aaj'
creating file '/tmp/lines-aak'
creating file '/tmp/lines-aal'
creating file '/tmp/lines-aam'
creating file '/tmp/lines-aan'
creating file '/tmp/lines-aao'
creating file '/tmp/lines-aap'
creating file '/tmp/lines-aaq'
creating file '/tmp/lines-aar'
creating file '/tmp/lines-aas'
creating file '/tmp/lines-aat'
creating file '/tmp/lines-aau'
creating file '/tmp/lines-aav'
creating file '/tmp/lines-aaw'
creating file '/tmp/lines-aax'
creating file '/tmp/lines-aay'
creating file '/tmp/lines-aaz'
creating file '/tmp/lines-aba'
creating file '/tmp/lines-abb'
creating file '/tmp/lines-abc'
creating file '/tmp/lines-abd'
creating file '/tmp/lines-abe'
creating file '/tmp/lines-abf'
creating

In [11]:
# line_cleaner is a function which brings all the characters to lowercase and removes some of the punctuations
def line_cleaner(line):
    line=line.lower()
    temp=""
    for char in line:
#       print(char)
        if char!= "," and char!= "!" and char!= "?" and char!= "." and char!= "-":
         if char!=',':
            temp+=char
    return temp

#example
line_cleaner("Where are you going? I am thinking of returning to the campus, but I- CANNOT!")

'where are you going i am thinking of returning to the campus but i cannot'

In [12]:
import re

def _should_skip(line, min_length, max_length):
    """Whether a line should be skipped depending on the length."""
    return len(line) < min_length or len(line) > max_length


In [13]:
def _preprocess_line(line):
    line = line.decode("utf-8")

    # Remove the first word if it is followed by colon (speaker names)
    # NOTE: this wont work if the speaker's name has more than one word
    line = re.sub('(?:^|(?:[.!?]\\s))(\\w+):', "", line)

    # Remove anything between brackets (corresponds to acoustic events).
    line = re.sub("[\\[(](.*?)[\\])]", "", line)

    # Strip blanks hyphens and line breaks
    line = line.strip(" -\n")

    return line

In [14]:
def create_example(previous_lines, line, file_id):
    """Creates examples with multi-line context
    The examples will include:
        file_id: the name of the file where these lines were obtained.
        response: the current line text
        context: the previous line text
        context/0: 2 lines before
        context/1: 3 lines before, etc.
    """
    example = {
        'file_id': file_id,
        'context': previous_lines[-1],
        'response': line,
    }
    example['file_id'] = file_id
    example['context'] = previous_lines[-1]

    extra_contexts = previous_lines[:-1]
    example.update({
        'context/{}'.format(i): context
        for i, context in enumerate(extra_contexts[::-1])
    })

    return example


In [15]:
def _create_examples_from_file(file_name, min_length=0, max_length=20,
                               num_extra_contexts=5):

    previous_lines = []
    with open(file_name, 'rb') as f:
      for line in f :
        line = _preprocess_line(line)
        if not line:
            continue

        should_skip = _should_skip(
            line,
            min_length=min_length,
            max_length=max_length)

        if previous_lines:
            should_skip |= _should_skip(
                previous_lines[-1],
                min_length=min_length,
                max_length=max_length)

            if not should_skip:
                yield create_example(previous_lines, line, file_name)

        previous_lines.append(line)
        if len(previous_lines) > num_extra_contexts + 1:
            del previous_lines[0]

In [16]:
example = _create_examples_from_file(file_name='/tmp/lines-gln')

count = 0
for i in example:
  count += 1
print('Found '+ str(count*5) + ' examples')
print(i)

Found 110985 examples
{'file_id': '/tmp/lines-gln', 'context': 'That was close!', 'response': 'We could have died!', 'context/0': "I can't go to sleep", 'context/1': "What's with all the noise?", 'context/2': "Hey how come this idiot didn't get hit?", 'context/3': 'What he said is true', 'context/4': 'Ha!'}


In [17]:
test = _create_examples_from_file(file_name='/tmp/lines-gln')
for x in test:
  print(x)

Streaming output truncated to the last 5000 lines.
{'file_id': '/tmp/lines-gln', 'context': 'trying to save him.', 'response': 'Me?', 'context/0': 'I was investigating a lead,', 'context/1': 'Bob and Carol.', 'context/2': 'Someone tried to kill you?', 'context/3': "I'm sorry.", 'context/4': "I'm..."}
{'file_id': '/tmp/lines-gln', 'context': 'No, no, it is...', 'response': 'Maybe.', 'context/0': 'You always saving me?', 'context/1': "That's what this is about, right?", 'context/2': 'Me?', 'context/3': 'trying to save him.', 'context/4': 'I was investigating a lead,'}
{'file_id': '/tmp/lines-gln', 'context': 'I know.', 'response': "I'm not blaming you.", 'context/0': 'Yeah, because we decided that we were rushing things.', 'context/1': "It's just, ever since we decided to postpone the wedding...", 'context/2': "I don't know.", 'context/3': "Come on, where's this coming from?", 'context/4': 'What?'}
{'file_id': '/tmp/lines-gln', 'context': 'Um, 9:00,', 'response': 'Zalman Plaza.', 'contex

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:

test = _create_examples_from_file(file_name='/tmp/lines-gln')
for x in test:
  print(x['context/0'])

KeyError: ignored

In [19]:
#test = _create_examples_from_file(file_name='/tmp/lines-gln')

inputs = []
responses = []
i = 0

for t in test:

  input_1 = t['context']
  response = t['response']
  #input_2 = test['context/0']
 # input_3 = test['context/1']
 # input_4 = test['context/2']
 # input_5 = test['context/3']
    
  inputs.append(line_cleaner(input_1))
 # inputs.append(line_cleaner(input_2))
 # inputs.append(line_cleanerr(input_3))
 # inputs.append(line_cleaner(input_4))
 # inputs.append(line_cleaner(input_5))
  responses.append('startsent' + ' ' +line_cleaner(response)+ ' ' + 'endsent')
    

  #for j in range(5):
    #responses.append('startsent' + ' ' +line_cleaner(response)+ ' ' + 'endsent')
    
  i += 5


In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

oov_token = "<OOV>"
max_length = 25

tokenizer = Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(inputs)
tokenizer.fit_on_texts(responses)

word_index = tokenizer.word_index
word_index['startsent'] = 0
word_index['endsent'] = len(word_index)+1
index_word = {word_index[word]:word for word in word_index}
print(index_word)
vocab_size = len(word_index) + 1
input_seq = tokenizer.texts_to_sequences(inputs)
response_seq = tokenizer.texts_to_sequences(responses)
input_seq_pad = pad_sequences(input_seq, maxlen = max_length ,padding = 'post', truncating = 'post')
response_seq_pad = pad_sequences(response_seq, maxlen = max_length, padding = 'post', truncating = 'post')

print(len(word_index))
print(inputs[0], responses[0])
print(input_seq_pad[0], response_seq_pad[0])
print(len(input_seq_pad))

{1: '<OOV>', 0: 'startsent', 4140: 'endsent', 4: 'you', 5: 'i', 6: 'it', 7: 'no', 8: 'what', 9: 'oh', 10: 'is', 11: "i'm", 12: '♪', 13: 'on', 14: 'me', 15: 'the', 16: 'yeah', 17: 'a', 18: 'okay', 19: "it's", 20: 'go', 21: 'my', 22: 'come', 23: 'not', 24: 'that', 25: 'to', 26: 'do', 27: 'hey', 28: 'this', 29: 'are', 30: 'yes', 31: 'know', 32: 'right', 33: 'we', 34: "don't", 35: 'here', 36: 'get', 37: "that's", 38: 'all', 39: 'sorry', 40: 'god', 41: 'good', 42: 'so', 43: 'in', 44: 'and', 45: 'just', 46: 'did', 47: "you're", 48: 'of', 49: 'stop', 50: 'your', 51: 'was', 52: 'up', 53: 'him', 54: 'have', 55: 'be', 56: 'hello', 57: 'thank', 58: "i'll", 59: 'he', 60: "what's", 61: 'can', 62: 'help', 63: 'out', 64: '¶', 65: 'why', 66: 'wait', 67: 'back', 68: 'for', 69: 'got', 70: 'how', 71: 'well', 72: 'shit', 73: "let's", 74: 'please', 75: 'there', 76: 'see', 77: 'look', 78: 'her', 79: 'going', 80: 'man', 81: 'where', 82: 'now', 83: "he's", 84: 'mom', 85: 'dad', 86: 'she', 87: 'fine', 88: 'wil

In [39]:
def preprocess(text_list):
  inputs = []
  for sent in text_list:
    inputs.append(line_cleaner(sent))
  input_seq = tokenizer.texts_to_sequences(inputs)
  input_seq_pad = pad_sequences(input_seq, maxlen = max_length ,padding = 'post', truncating = 'post')
  return input_seq_pad

In [22]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt

--2020-06-24 20:52:58--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 2a00:1450:4013:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘/tmp/glove.6B.100d.txt’

/tmp/glove.6B.100d. 100%[===================>] 331.04M  77.4MB/s    in 4.3s    

2020-06-24 20:53:03 (77.4 MB/s) - ‘/tmp/glove.6B.100d.txt’ saved [347116733/347116733]



In [23]:
import os
import numpy as np

embeddings_index = {} #initialize dictionary
f = open('/tmp/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [24]:
embedding_dim = 100

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector


In [25]:
     embeddings_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.74746001, -0.36522999,  0.22048   , ..., -0.30627999,
         0.43299001,  0.015122  ],
       [-0.031714  , -0.099512  ,  0.35916001, ..., -0.23621   ,
         0.17083   ,  0.62533998],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [26]:
print(len(embeddings_matrix))
embeddings_matrix[7]

4141


array([-0.013767  ,  0.33247   ,  0.59895003, -0.51560998, -0.56200999,
        0.12329   , -0.13417   ,  0.26010999,  0.65116   ,  0.024565  ,
        0.31990001,  0.49118   ,  0.30761999,  0.53333002,  0.68014997,
       -0.25419   , -0.068138  ,  0.26390001, -0.35962999,  0.47576001,
        0.36392   ,  0.23683999, -0.24312   , -0.52683997,  0.15305001,
        0.032089  , -0.11053   , -0.71643001, -0.026425  , -0.41872001,
        0.18218   ,  0.084099  , -0.18880001,  0.22899   ,  0.30495   ,
        0.45337   ,  0.27868   ,  0.054886  , -0.046348  ,  0.14313   ,
       -0.48341   ,  0.27654001,  0.53847998, -0.66876   ,  0.13568   ,
       -0.45659   ,  0.20602   , -0.67056   , -0.65925997, -1.09109998,
        0.24557   , -0.14213   ,  0.086415  ,  0.85842001, -0.016081  ,
       -2.7815001 ,  0.40608001, -0.094489  ,  1.77760005,  0.85031998,
       -0.34224001,  0.39772001, -0.83965999,  0.13606   ,  1.18959999,
       -0.17331   ,  0.71847999,  0.042783  , -0.24022   ,  0.07

In [27]:
print(len(input_seq_pad))
input_seq_pad

22196


array([[ 922,  290, 2379, ...,    0,    0,    0],
       [  65,   23,    0, ...,    0,    0,    0],
       [2380,    0,    0, ...,    0,    0,    0],
       ...,
       [ 163,    0,    0, ...,    0,    0,    0],
       [   5,   97,   20, ...,    0,    0,    0],
       [  24,   51,  152, ...,    0,    0,    0]], dtype=int32)

In [28]:
print(len(response_seq_pad))
response_seq_pad

22196


array([[   0, 3065, 4140, ...,    0,    0,    0],
       [   0, 1900,    4, ...,    0,    0,    0],
       [   0,    6, 1250, ...,    0,    0,    0],
       ...,
       [   0,    8,   59, ...,    0,    0,    0],
       [   0,   24,   51, ...,    0,    0,    0],
       [   0,   33,  287, ...,    0,    0,    0]], dtype=int32)

In [29]:
X_train = []
y_train = []
for j in range(22195):
    X_train.append(input_seq_pad[j])
    y_train.append(response_seq_pad[j])

X_train = np.array(X_train)
y_train = np.array(y_train)

print(embedding_dim)
print(X_train.shape)
print(y_train.shape)

dataset = (inputs, responses)

100
(22195, 25)
(22195, 25)


In [30]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, GRU, Dense 

max_sequence_len = 25
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(1024)
train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [31]:
class Encoder(tf.keras.Model):
    def __init__(self, hidden_size=1024, max_sequence_len=25, batch_size=batch_size, embedding_dim=100, vocab_size=vocab_size+1):
        super(Encoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.vocab_size = vocab_size
        self.max_sequence_len = max_sequence_len
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.embedding_layer = Embedding(
            input_dim=self.vocab_size, output_dim=self.embedding_dim, weights=[embeddings_matrix], trainable=False)
        self.GRU_1 = GRU(units=hidden_size, return_sequences=True,recurrent_initializer='glorot_uniform')
        self.GRU_2 = GRU(units=hidden_size,
                         return_sequences=True, return_state=True,recurrent_initializer='glorot_uniform')

    def initial_hidden_state(self):
        return tf.zeros(shape=(self.batch_size, self.hidden_size))

    def call(self, x, initial_state, training=False):
        x = self.embedding_layer(x)
        x = self.GRU_1(x, initial_state=initial_state)
        x, hidden_state = self.GRU_2(x)
        return x, hidden_state

In [32]:
class Attention(tf.keras.Model):
    def __init__(self, hidden_size=256):
        super(Attention, self).__init__()
        self.fc1 = Dense(units=hidden_size)
        self.fc2 = Dense(units=hidden_size)
        self.fc3 = Dense(units=1)

    def call(self, encoder_output, hidden_state, training=False):
        '''hidden_state : h(t-1)'''
        y_hidden_state = tf.expand_dims(hidden_state, axis=1)
        y_hidden_state = self.fc1(y_hidden_state)
        y_enc_out = self.fc2(encoder_output)

        y = tf.keras.backend.tanh(y_enc_out + y_hidden_state)
        attention_score = self.fc3(y)
        attention_weights = tf.keras.backend.softmax(attention_score, axis=1)

        context_vector = tf.multiply(encoder_output, attention_weights)
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [33]:
class Decoder(tf.keras.Model):
    def __init__(self, hidden_size=1024, max_sequence_len=25, batch_size=batch_size, embedding_dim=100, vocab_size=vocab_size+1):
        super(Decoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.vocab_size = vocab_size
        self.max_sequence_len = max_sequence_len
        self.hidden_size = hidden_size
        self.batch_size = batch_size
    
        self.embedding_layer = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, weights=[embeddings_matrix], trainable=False)
        self.GRU = GRU(units=hidden_size,
                       return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.attention = Attention(hidden_size=self.hidden_size)
        self.fc = Dense(units=self.vocab_size)

    def initial_hidden_state(self):
        return tf.zeros(shape=(self.batch_size, self.hidden_size))

    def call(self, x, encoder_output, hidden_state, training=False):
        x = self.embedding_layer(x)
        context_vector, attention_weights = self.attention(
            encoder_output, hidden_state, training=training)
        contect_vector = tf.expand_dims(context_vector, axis=1)
        x = tf.concat([x, contect_vector], axis=-1)
        x, curr_hidden_state = self.GRU(x)
        x = tf.reshape(x, shape=[self.batch_size, -1])
        x = self.fc(x)
        return x, curr_hidden_state, attention_weights

Defining training loop, loss function and optimizer

In [34]:
loss_object = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
train_accuracy = tf.metrics.SparseCategoricalAccuracy()

def loss_function(y_true, y_pred):
    loss = loss_object(y_true, y_pred)
    mask = 1 - tf.cast(tf.equal(y_true, 0), 'float32')
    return tf.reduce_mean(loss * mask)

In [35]:
def training_step(inputs, responses):    
    with tf.GradientTape() as Tape:
        encoder_init_state = encoder.initial_hidden_state()
        encoder_output, encoder_hidden_state = encoder(inputs, encoder_init_state, training=True)
        decoder_hidden = encoder_hidden_state
        loss = 0
        acc = []
        current_word = tf.expand_dims(responses[:, 0], axis=1)
        for word_idx in range(1, max_sequence_len):
            next_word = responses[:, word_idx]
            logits, decoder_hidden, attention_weights = decoder(current_word, encoder_output, decoder_hidden)
            loss += loss_function(next_word, logits)
            acc.append(train_accuracy(next_word, logits))
            current_word = tf.expand_dims(next_word, axis=1)
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = Tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return loss, tf.reduce_mean(acc)

encoder = Encoder()
decoder = Decoder()

Training the model

In [36]:
epochs = 20
NUM_SAMPLES = len(X_train)
num_steps = NUM_SAMPLES // batch_size

for epoch in range(1, epochs + 1):
    print(f'Epoch {epoch}/{epochs}')
    ep_loss = []
    ep_acc = []
    progbar = tf.keras.utils.Progbar(target=num_steps, stateful_metrics=[
                                     'curr_loss', 'curr_accuracy'], unit_name='batch')

    for step, example in enumerate(train_dataset):
        inputs = example[0]
        responses = example[1]
        loss, acc = training_step(inputs, responses)
        loss /= responses.shape[1]
        ep_loss.append(loss)
        ep_acc.append(acc)
        progbar.update(
            step + 1, values=[('curr_loss', loss), ('curr_accuracy', acc)])

    print(f'Metrics after epoch {epoch} : Loss => {np.mean(ep_loss):.3f} | Accuracy => {np.mean(ep_acc):.3f}')

Epoch 1/20
346/346 [==============================] - 168s 486ms/batch - curr_loss: 0.4549 - curr_accuracy: 0.8715
Metrics after epoch 1 : Loss => 0.554 | Accuracy => 0.772
Epoch 2/20
346/346 [==============================] - 161s 466ms/batch - curr_loss: 0.3829 - curr_accuracy: 0.8889
Metrics after epoch 2 : Loss => 0.476 | Accuracy => 0.882
Epoch 3/20
346/346 [==============================] - 162s 467ms/batch - curr_loss: 0.3438 - curr_accuracy: 0.8953
Metrics after epoch 3 : Loss => 0.443 | Accuracy => 0.892
Epoch 4/20
346/346 [==============================] - 162s 467ms/batch - curr_loss: 0.3188 - curr_accuracy: 0.8991
Metrics after epoch 4 : Loss => 0.410 | Accuracy => 0.897
Epoch 5/20
346/346 [==============================] - 161s 467ms/batch - curr_loss: 0.2420 - curr_accuracy: 0.9021
Metrics after epoch 5 : Loss => 0.375 | Accuracy => 0.900
Epoch 6/20
346/346 [==============================] - 162s 468ms/batch - curr_loss: 0.1982 - curr_accuracy: 0.9051
Metrics after epoch 

In [37]:
def give_response(sentence):
    sentence = preprocess([sentence])
    enc_init = tf.zeros(shape=[1, 1024])
    enc_out, enc_hidden = encoder(sentence, enc_init)
    decoder.batch_size = 1
    tokenizer.index_word[0] = ''
    decoded = []
    att = []
    current_word = tf.expand_dims([word_index['startsent']], axis=0) 
    decoder_hidden = enc_hidden
    for word_idx in range(1, max_sequence_len):
        logits, decoder_hidden, attention_weights = decoder(current_word, enc_out, decoder_hidden)
        decoded_idx = np.argmax(logits)
        if index_word[decoded_idx] == 'endsent':
            break
        decoded.append(tokenizer.index_word[decoded_idx])
        att.append(attention_weights.numpy().squeeze())
        current_word = tf.expand_dims([decoded_idx], axis=0)
    return ' '.join(decoded), att

In [43]:
sentences = ['who are you ?',
             'do you miss college?',
             'It is going to be hard',
             'I like this',
             'how are you doing ?', 
             'help',
             'Are you sad?']
for inp_sentence in sentences:
    inp_array = inp_sentence.split()
    inp_len = len(inp_sentence.split())
    trans_sentence, attention_weights = give_response(inp_sentence)
    trans_array = trans_sentence.split()
    trans_len = len(trans_array)
    print('INPUT : ', inp_sentence)
    print('RESPONSE : ', trans_sentence)
    print('-'*30)

INPUT :  who are you ?
RESPONSE :  mmy name is sorbet
------------------------------
INPUT :  do you miss college?
RESPONSE :  yes sir
------------------------------
INPUT :  It is going to be hard
RESPONSE :  hold him
------------------------------
INPUT :  I like this
RESPONSE :  yeah
------------------------------
INPUT :  how are you doing ?
RESPONSE :  hello
------------------------------
INPUT :  help
RESPONSE :  she's stuck
------------------------------
INPUT :  Are you sad?
RESPONSE :  are you know
------------------------------
